In [1]:
import pymupdf
from langchain_community.document_loaders import PyMuPDFLoader

In [42]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")


In [3]:
# # To get the books index or Table of content (TOC)
# pdf = pymupdf.open(pdf_path)
# toc = pdf.get_toc() # To acces the TOC

# if toc:
#     for entry in toc:
#         level, title, page = entry
#         print(f"Level {level} | {title} | Page{page}")

# else:
#     print("No Table of Content found")

Level → The depth or hierarchy of the section.
Eg: Level 2 - Chapters
    Level 3 - Main topics
    Level 4 - Subtopics of main topics

In [4]:
from langchain_docling import DoclingLoader

pdf_path = "/Users/adityasingh/Desktop/LLM_projects/AI_Tutor/backend/data/Hands-on-ML.pdf"

loader = DoclingLoader(pdf_path)
docs = loader.load()

2025-10-13 14:52:41,532 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-13 14:52:41,654 - INFO - Going to convert document batch...
2025-10-13 14:52:41,655 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-13 14:52:41,674 - INFO - Loading plugin 'docling_defaults'
2025-10-13 14:52:41,675 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-10-13 14:52:41,675 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-13 14:52:41,682 - INFO - Loading plugin 'docling_defaults'
2025-10-13 14:52:41,684 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-10-13 14:52:41,684 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-13 14:52:43,809 - INFO - Accelerator device: 'mps'
2025-10-13 14:52:46,809 - I

In [5]:
docs[0]

Document(metadata={'source': '/Users/adityasingh/Desktop/LLM_projects/AI_Tutor/backend/data/Hands-on-ML.pdf', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/1', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 36.0, 't': 372.5000502929687, 'r': 358.776, 'b': 326.4610502929687, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 60]}]}], 'headings': ['Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow'], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 15959967756236519746, 'filename': 'Hands-on-ML.pdf'}}}, page_content='Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow\nConcepts, Tools, and Techniques to Build Intelligent Systems')

In [6]:
import pprint 
pprint.pprint(docs[0].metadata)

{'dl_meta': {'doc_items': [{'children': [],
                            'content_layer': 'body',
                            'label': 'text',
                            'parent': {'$ref': '#/body'},
                            'prov': [{'bbox': {'b': 326.4610502929687,
                                               'coord_origin': 'BOTTOMLEFT',
                                               'l': 36.0,
                                               'r': 358.776,
                                               't': 372.5000502929687},
                                      'charspan': [0, 60],
                                      'page_no': 1}],
                            'self_ref': '#/texts/1'}],
             'headings': ['Hands-on Machine Learning with Scikit-Learn, Keras '
                          '& TensorFlow'],
             'origin': {'binary_hash': 15959967756236519746,
                        'filename': 'Hands-on-ML.pdf',
                        'mimetype': 'application/pdf'}

In [16]:
from docling.document_converter import DocumentConverter

converter = DocumentConverter()
# Convert the PDF to a Docling document
doc = converter.convert(pdf_path).document

# Export the document to Markdown
full_markdown_content = doc.export_to_markdown()

2025-10-13 15:50:01,523 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-13 15:50:01,635 - INFO - Going to convert document batch...
2025-10-13 15:50:01,639 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-10-13 15:50:01,639 - INFO - Accelerator device: 'mps'
2025-10-13 15:50:07,747 - INFO - Accelerator device: 'mps'
2025-10-13 15:50:08,999 - INFO - Accelerator device: 'mps'
2025-10-13 15:50:09,697 - INFO - Processing document Hands-on-ML.pdf
2025-10-13 16:27:02,422 - INFO - Finished converting document Hands-on-ML.pdf in 643.62 sec.


In [17]:
full_markdown_content

'<!-- image -->\n\n## Hands-on Machine Learning with Scikit-Learn, Keras &amp; TensorFlow\n\nConcepts, Tools, and Techniques to Build Intelligent Systems\n\n<!-- image -->\n\n<!-- image -->\n\n## OREILLY\n\n## Hands-On Machine Learning with Scikit-Learn Keras, and TensorFlow\n\nThrough a series of breakthroughs, Deep Learning has boosted the entire field of machine learning. Now; even programmers who know close to nothing about this technology can use sim ple, efficient tools to implement programs capable of learning from data. The updated edition of this best-selling book uses concrete examples, minimal theory; and production-ready Python frameworks to help you an intuitive understanding of the concepts and tools for building intelligent systems. gain\n\nYou\'Il learn a range of techniques that you can quickly to use. With exercises in each chapter to help you apply what you\'ve learned; all you need is programming experience to get started. All code is available on GitHub. It has bee

Now we have a fully markdown form of our book with table and placeholders of the image. Now we can use `MarkdownHeaderTextSplitter` to split our document

In [18]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Defining the header we want to split
headers_to_split_on = [
    ("## PART", "Part"),
    ("## CHAPTER", "Chapter"),
    ("## ", "Section"),
]
# Creating the splitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on  = headers_to_split_on,
    strip_headers = False
)

# Creating the chunks
chunks = markdown_splitter.split_text(full_markdown_content)
print(chunks)

[Document(metadata={}, page_content='<!-- image -->  \n## Hands-on Machine Learning with Scikit-Learn, Keras &amp; TensorFlow  \nConcepts, Tools, and Techniques to Build Intelligent Systems  \n<!-- image -->  \n<!-- image -->  \n## OREILLY  \n## Hands-On Machine Learning with Scikit-Learn Keras, and TensorFlow  \nThrough a series of breakthroughs, Deep Learning has boosted the entire field of machine learning. Now; even programmers who know close to nothing about this technology can use sim ple, efficient tools to implement programs capable of learning from data. The updated edition of this best-selling book uses concrete examples, minimal theory; and production-ready Python frameworks to help you an intuitive understanding of the concepts and tools for building intelligent systems. gain  \nYou\'Il learn a range of techniques that you can quickly to use. With exercises in each chapter to help you apply what you\'ve learned; all you need is programming experience to get started. All cod

In [19]:
for chunk in chunks[:40]:
    print("Part:", chunk.metadata.get("Part"))
    print("Chapter:", chunk.metadata.get("Chapter"))
    print("Section:", chunk.metadata.get("Section"))
    print("Content snippet:", chunk.page_content[:50])
    print("------")

Part: None
Chapter: None
Section: None
Content snippet: <!-- image -->  
## Hands-on Machine Learning with
------
Part: I
Chapter: None
Section: None
Content snippet: ## PART I  
## The Fundamentals of Machine Learnin
------
Part: None
Chapter: 1 The Machine Learning Landscape
Section: None
Content snippet: ## CHAPTER 1 The Machine Learning Landscape  
When
------
Part: None
Chapter: 2 End-to-End Machine Learning Project
Section: None
Content snippet: ## CHAPTER 2 End-to-End Machine Learning Project  
------
Part: None
Chapter: 3
Section: None
Content snippet: ## CHAPTER 3  
## Classification  
In  Chapter  1 
------
Part: None
Chapter: 4 Training Models
Section: None
Content snippet: ## CHAPTER 4 Training Models  
So far we have trea
------
Part: None
Chapter: 5 Support Vector Machines
Section: None
Content snippet: ## CHAPTER 5 Support Vector Machines  
A Support  
------
Part: None
Chapter: 7 Ensemble Learning and Random Forests
Section: None
Content snippet: ## CHAPTER 7 Ensemble L

In [20]:
print(chunks[11].metadata)

{'Chapter': '10 Introduction to Artificial Neural Networks'}


In [21]:
print(chunks[11].page_content)

## CHAPTER 10 Introduction to Artificial Neural Networks  
## with Keras  
Birds  inspired  us  to  fly,  burdock  plants  inspired  Velcro,  and  nature  has  inspired countless more inventions. It seems only logical, then, to look at the brain's architecture  for  inspiration  on  how  to  build  an  intelligent  machine.  This  is  the  logic  that sparked artificial  neural  networks (ANNs):  an  ANN  is  a  Machine  Learning  model inspired  by  the  networks  of  biological  neurons  found  in  our  brains.  However, although planes were inspired by birds, they don't have to flap their wings. Similarly, ANNs  have  gradually  become  quite  different  from  their  biological  cousins.  Some researchers even argue that we should drop the biological analogy altogether (e.g., by saying  'units'  rather  than  'neurons'),  lest  we  restrict  our  creativity  to  biologically plausible systems. 1  
ANNs are at the very core of Deep Learning. They are versatile, powerful, and scalable

### Embedding and Vector store

In [22]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings

model = "BAAI/bge-small-en-v1.5"

import os

# Initializing the embedding model
embedding_model = HuggingFaceEndpointEmbeddings(
    model=model
)

In [23]:
# Creating vector store
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name = "Hands_on_ML",
    embedding_function = embedding_model,
    persist_directory="/Users/adityasingh/Desktop/LLM_projects/AI_Tutor/backend/data/vector_store/chroma_db_store",
)

In [24]:
# Add documents
vector_store.add_documents(documents=chunks)

['eebc63e7-8bde-479c-8b07-2503543626af',
 '441346dc-b7ab-4c4c-a343-b0d00bb6fe79',
 '642e2a5b-3d83-451a-9355-39e1147a4cb4',
 '24bbb177-19fa-49b2-872e-fbfa02445f26',
 '6db08fcc-6dd1-41c7-bd97-43c7a6e62e58',
 '4b117eaf-ed55-4356-9867-4a27cbd2a845',
 'cbbd0769-3b3d-4a85-abb5-aa87fd2a2504',
 '7c2b7eb2-89ab-4c34-af4b-749727b5fd06',
 'eb6c7d3f-a9ae-477e-a372-378bc977f75f',
 'aca6d9b6-d334-40f9-aa4c-2c11585d9e05',
 'ddef7df5-18bc-4ed9-a412-c1cdb42e3099',
 '5f00eb3e-7912-4230-9dbb-ab4ab198b3b7',
 'bd0d0601-e3f3-4643-84b0-182d20cbc4ce',
 'd1d6036e-af71-49fb-a6e9-7ff1480c1852',
 '2d8450c9-202d-4905-a43d-720768c219dd',
 '5271ff54-485f-4f77-8ac2-cb4f8d8896cf',
 'd565d889-cce2-4db9-8257-5d3438709f9f',
 '107943b3-4c0c-4e92-bcd3-45833d508146',
 '26ec7ca6-1f67-4aae-872c-b18637f628e7',
 'b7a7c978-7cbb-4a61-83cc-55ea1f1a8427',
 '87e7c483-461f-48ea-a07e-26cd00aff1ef']

In [25]:
query = "What is a confusion matrix"
results = vector_store.similarity_search(query, k=3)
print(results[0])

page_content='## Confusion Matrix  
A much better way to evaluate the performance of a classifier is to look at the confusion matrix . The general idea is to count the number of times instances of class A are classified as class B. For example, to know the number of times the classifier confused images of 5s with 3s, you would look in the fifth row and third column of the confusion matrix.  
To compute the confusion matrix, you first need to have a set of predictions so that they can be compared to the actual targets. You could make predictions on the test set, but let's keep it untouched for now (remember that you want to use the test set only at the very end of your project, once you have a classifier that you are ready to launch). Instead, you can use the cross\_val\_predict() function:  
```
from sklearn.model_selection import cross_val_predict y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
```  
Just  like  the cross\_val\_score() function, cross\_val\_predict

Retrieval Pipeline of `RAG` is done till here... Now, the generation part is left.

In [26]:
# Creating retriever
retriever = vector_store.as_retriever(search_kwargs={"k":3})

### LLM Integration

In [27]:
load_dotenv()

True

In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Using Gemini 2.5 flash model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0.4,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

### Prompt Template

In [29]:
# Design chat prompt template
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
"""
You are an expert AI Tutor with a talent for making complex concepts engaging and easy to understand. 

**Your goals:**
- Teach the student as if in a real classroom, guiding them step-by-step.
- Start with a **clear and concise definition** of the concept.
- Expand into a **deep explanation with examples, analogies, and real-world use cases.**
- If relevant, include **mathematical intuition, diagrams (describe them in words), or historical context.**
- Finish with a **summary** and a question back to the student to check understanding.

**Constraints:**
- Use the provided context as the main source of truth.
- If the context is insufficient, clearly state that and guide the student forward.

**Context:**
---
{context}
---

**Student's Question:** {question}
""")

2025-10-13 18:10:52,842 - INFO - Backing off send_request(...) for 0.4s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))


In [31]:
# Building RAG chain with Gemini LLM
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [32]:
question = "Explain me neural network in detail... how it works, what can they do?"
print(f"**Question:** {question}\n")

answer = rag_chain.invoke(question)
print(f"**Answer:** {answer}")

**Question:** Explain me neural network in detail... how it works, what can they do?

**Answer:** Alright class, settle in! Today, we're going to unravel one of the most fascinating and powerful concepts in Machine Learning: **Artificial Neural Networks (ANNs)**, often just called "neural networks."

Think of me as your guide through this journey. We'll start with the basics, then dive deep into how these incredible systems learn and what amazing things they can accomplish.

---

### What is an Artificial Neural Network?

At its core, an **Artificial Neural Network (ANN)** is a Machine Learning model inspired by the networks of biological neurons found in our brains. Just as our brains process information to learn and make decisions, ANNs are designed to learn complex patterns from data and make predictions or classifications.

While they draw inspiration from biology, it's important to remember that ANNs have evolved to be quite different from their biological cousins. Think of it lik

### Making our chatbot aware with conversation history(Convertional RAG chain)
Here we are using the old prompt only but with providing it the chat history for better answering and tracking of the lessons taught.


In [33]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.chains import create_history_aware_retriever

# Stores chat history for one conversation session
chat_history = InMemoryChatMessageHistory()

# History-aware retriever
rephrase_prompt = ChatPromptTemplate.from_messages([
    ("placeholder", "{chat_history}"),
    ("user", "{input}"),
    ("user", "Given the above conversation generate a query to generate a search query to look up in order to get information relevant to the conversation"), 
])

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Creating history aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    rephrase_prompt
)

In [34]:
# Final prompt that answers the question
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     
        You are an expert AI Tutor with a talent for making complex concepts engaging and easy to understand. 
        
        **Your goals:**
        - Teach the student as if in a real classroom, guiding them step-by-step.
        - Start with a **clear and concise definition** of the concept.
        - Expand into a **deep explanation with examples, analogies, and real-world use cases.**
        - If relevant, include **mathematical intuition, diagrams (describe them in words), or historical context.**
        - Finish with a **summary** and a question back to the student to check understanding.
        
        **Constraints:**
        - Use the provided context as the main source of truth.
        - If the context is insufficient, clearly state that and guide the student forward.
        
        **Context:**
        ---
        {context}
        ---
        
        **Student's Question:** {input} 
        
    """),

    ("placeholder", "{chat_history}"),
    ("user", "{input}"),
])

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

document_chain = create_stuff_documents_chain(llm, final_prompt)

# Final coversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever, 
    document_chain
)

In [36]:
# Function to ask question
def ask_tutor(question, chat_history):
    response = conversational_rag_chain.invoke({
        "chat_history": chat_history.messages,
        "input": question,
    })
    return response["answer"]

# usage
input1 = "What is difference between batch and online learning?"
reply = ask_tutor(input1, chat_history)

print(f"**You:** {input1}")
print(f"\n**Tutor**: {reply}")

# Adding the questions and response in history 
chat_history.add_user_message(input1)
chat_history.add_ai_message(reply)

# Asking 2nd follow-up question
input2 = "Can you explain that more simply?"
reply = ask_tutor(input2, chat_history)

print("\n--- Turn 2 (Follow-up) ---")
print(f"**You:** {input2}")
print(f"\n**Tutor:** {reply}")

chat_history.add_user_message(input2)
chat_history.add_ai_message(reply)

# Asking 3rd follow-up question
input3 = "Can you explain that more simply?"
reply = ask_tutor(input3, chat_history)

print("\n--- Turn 3 (Follow-up) ---")
print(f"**You:** {input3}")
print(f"\n**Tutor:** {reply}")

chat_history.add_user_message(input3)
chat_history.add_ai_message(reply)

**You:** What is difference between batch and online learning?

**Tutor**: That's a fantastic question, and it gets right to the heart of how Machine Learning systems can adapt and evolve! Let's break down Batch Learning and Online Learning step by step.

### What is Batch Learning?

Imagine you're preparing for a big final exam. You gather *all* your textbooks, notes, and practice problems, and you study everything at once before the exam. That's essentially how **Batch Learning** works.

**Definition:** In Batch Learning, a machine learning system is trained using **all available data at once**. It's like a one-shot learning process where the system processes the entire dataset to build its model.

**How it works:**
1.  **Offline Training:** This training typically happens "offline," meaning it's done in a controlled environment before the system is put into action. It takes a significant amount of time and computing power because it's crunching through a potentially massive dataset.

In [ ]:
# Que-1
input1 = "What are the main challenges of Machine Learning?"
reply = ask_tutor(input1, chat_history)

print(f"\n**You:** {input1}")
print(f"\n**Tutor**: {reply}")

# Adding the questions and response in history 
chat_history.add_user_message(input1)
chat_history.add_ai_message(reply)

# Que-2
input1 = "Can you tell me more about the first one you mentioned?"
reply = ask_tutor(input1, chat_history)

print(f"\n**You:** {input1}")
print(f"\n**Tutor**: {reply}")

# Adding the questions and response in history 
chat_history.add_user_message(input1)
chat_history.add_ai_message(reply)

# Que-3
input1 = "Why is that such a big problem for a model?"
reply = ask_tutor(input1, chat_history)

print(f"\n**You:** {input1}")
print(f"\n**Tutor**: {reply}")

# Adding the questions and response in history 
chat_history.add_user_message(input1)
chat_history.add_ai_message(reply)

### Making this a tool

In [40]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_react_agent, AgentExecutor

# Tool-1
textbook_tool = create_retriever_tool(
    history_aware_retriever,
    name = "ml_textbook_search",
    description = "Use this tool to answer questions about the uploaded book"
)

### Creating a websearch tool:
`SerpAPI`

In [44]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool

search = SerpAPIWrapper()

web_search_tool = Tool(
    name = "web_search",
    func = search.run,
    description = "A useful function for searching the internet about current events, real-world examples, or topics not foiund in user's uploded document." 
)

In [ ]:
# Assembling toolbox
tools = [textbook_tool, web_search_tool]

# Agent prompt
prompt = hub.pull("hwchase17/react-chat")
